In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, chdir, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from datetime import date, timedelta, datetime
from os.path import dirname, join, isfile, isdir
from bulletin import default_input, default_output
from bulletin.utils import utils, static, normalize

path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [ ]:

utils.create_backup(first_name = "backup_notifica_diario_" , level=3)
chdir(current_dir)

In [ ]:
from bulletin.utils.clean_up import clear_directories
clear_directories()

In [ ]:
today = date.today()
someday = date(2020, 1, 2)
diff = today - someday

In [ ]:
# COLUNAS DO BANCO DO GAL

# Index(['id'0, 'origem_gal'1, 'requisicao'2, 'data_requisicao'3, 'data_solicitacao'4,
#         'requisicao_correlativo'5, 'unidade_requisitante'6,
#         'unidade_requisitante_municipio'7, 'unidade_requisitante_uf'8, 'paciente'9,
#         'cns'10, 'cpf'11, 'nome_mae'12, 'sexo'13, 'data_nascimento'14, 'idade'15,
#         'raca_cor'16, 'idade_gestacional'17, 'ibge'18, 'municipio'19, 'latitude'20,
#         'longitude'21, 'codigo_regional'22, 'regional'23, 'data_inicio_sintomas'24,
#         'cod_exame_requisicao'25, 'data_coleta'26, 'codigo_amostra'27,
#         'numero_amostra'28, 'numero_interno'29, 'material_biologico'30,
#         'data_encaminhamento_amostra'31, 'data_recebimento_amostra'32,
#         'tempo_transporte_amostra'33, 'envio_oportuno'34, 'exame'35, 'metodo'36,
#         'resultado'37, 'virus'38, 'data_liberacao'39, 'laboratorio_executor'40,
#         'laboratorio_executor_municipio'41, 'correlativo_exame'42,
#         'tempo_liberacao'43, 'tempo_diagnostico'44, 'codigo_status'45, 'status'46,
#         'classificacao'47],
#        dtype='object')]

In [ ]:
gal_2020 = pd.read_pickle(join(path_file, 'banco_gal_2020-bz2.pkl'), compression='bz2')
gal_2020 = gal_2020[['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]

In [ ]:
gal_2021 = pd.read_pickle(join(path_file, 'banco_gal_2021-bz2.pkl'), compression='bz2')
gal_2021 = gal_2021[['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]

In [ ]:
gal_2021_complemento = pd.read_csv(join(path_file, 'consulta-gal-complemento-2021.csv'), sep = ';', usecols=['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao'])

In [ ]:
gal = pd.concat([gal_2020, gal_2021, gal_2021_complemento])

In [ ]:
from epiweeks import Week

gal['data_requisicao'] = pd.to_datetime(gal['data_requisicao'])
gal['se'] = gal['data_requisicao'].apply(lambda x: Week.fromdate(x).week)
gal['ano'] = gal['data_requisicao'].dt.year
gal['mes'] = gal['data_requisicao'].dt.month
gal['dia'] = gal['data_requisicao'].dt.day

In [ ]:
lab_estado = ['INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA - IBMP','IBMP','INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA','LABORATORIO CENTRAL DO ESTADO','LACEN']
gal['laboratorio_executor'] = gal['laboratorio_executor'].apply(normalize.normalize_text)

In [ ]:
amostra_por_dia = gal.loc[gal['laboratorio_executor'].isin(lab_estado)].groupby(['ano','mes','se','dia','classificacao'])[['id']].count().unstack('classificacao').fillna(0).astype(int).droplevel(axis=1, level=0)
amostra_por_dia['total'] = amostra_por_dia[['Confirmado','Descartado']].sum(1).astype(int)
amostra_por_dia['positividade'] = amostra_por_dia['Confirmado'] / amostra_por_dia['total']
amostra_por_dia

In [ ]:
ax = amostra_por_dia[['Confirmado','Descartado', 'total']].rolling(7).mean().plot()
amostra_por_dia[['positividade']].rolling(7).mean().plot(secondary_y=True, ax=ax, ls=':', figsize=(20,12))

In [ ]:
amostra_por_dia.to_excel('teste_gal_amostra_positividade_dia.xlsx')

In [ ]:
amostra_por_dia.